# 클러스터별 2개씩, 최종 18개 종목 선정하기
- 거래대금(Amount) 기준


In [ ]:
# 코랩에서 실행시 pyMySQL을 먼저 설치해야 한다
!pip install JPype1
#!pip install konlpy
!pip install pyMySQL
!pip install -U finance-datareader
!pip install attention

In [ ]:
#-- Import packages --
import os
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
import sqlite3
from sqlalchemy import create_engine
#import konlpy
#from konlpy.tag import Okt
import pymysql
from os import replace
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from datetime import datetime
from sqlalchemy import create_engine
import ast
from os import replace
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
import re
import string
import itertools
import warnings
warnings.filterwarnings('ignore')
#-- Import packages :END --

In [ ]:
######################################################################

In [ ]:
#---------------------------------------------------------------------
# 해당 클러스터 안의 종목 리스트 가져오기
#-- get_stock_from_cluster(c_number) --------------------------------
def get_stock_from_cluster(c_number):
    # DB 연결 설정
    conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
    # 읽어오기 위한 sql 준비
    sql = f"SELECT x.* FROM cluster_{c_number} x"
    # DB 검색결과를 dataframe에 저장
    df_stocks = pd.read_sql_query(sql, conn)
    # DB 연결 close
    conn.close()
    # 컬럼 정리
    df_stocks.columns = ['Code','serial_number', 'cluster_labels']
    # 값 리턴
    return df_stocks
#-- get_stock_from_cluster(c_number) : End --------------------------

#--------------------------------------------------------------------
# 상위 18개 종목 DB에 저장
#-- save_top_18(df_top_18) ------------------------------------------
def save_top_18(df_top_18):
  # 테이블 생성 및 연결
  # DB 저장 연결 준비
  db_connection_path = 'mysql+pymysql://아이디:패스워드@호스트주소:포트번호/DB이름'
  # DB 커넥션 생성
  db_connection = create_engine(db_connection_path)
  # 데이터프레임을 DB로 저장, 이미 있으면 덮어쓰기(replace)
  df_top_18.to_sql(name='top_18_by_Amount', con=db_connection,if_exists='replace',index=False)
  # DB close
  db_connection.dispose()
#-- save_top_18(df_top_18) : End------------------------------------
#--------------------------------------------------------------------

In [ ]:
#====================================================================
# 메인 함수 실행 부분
#--------------------------------------------------------------------
# FinaceReader로 종목 정보 가져오기 시총 기준 리스트
stocks = fdr.StockListing('KRX')
# 작업 df로 copy()
df_stocks = stocks.copy()
# 작업 df의 컬럼 정리
df_stocks_amt_rank = df_stocks[['Code','Name','Amount','Marcap','Stocks']]
# 거래대금 (Amount) 기준으로 정렬
df_stocks_amt_rank = df_stocks_amt_rank.sort_values('Amount', ascending=False)
# 정렬순서로 인덱스 리셋
df_stocks_amt_rank.reset_index(inplace=True)
# 상위18개 종목 저장위한 df 생성
df_18_selection = pd.DataFrame()
# 9개 클러스터 대상 반복
for i in range(9):
    # 해당 클러스터의 종목 리스트 가져오기
    df_cluster = get_stock_from_cluster(i)
    # 거래대금 기준 정렬된 KRX 리스트와 inner join
    df_merged= pd.merge(df_stocks_amt_rank, df_cluster, on='Code', how='inner')
    # 확인을 위해 다시 거래대금(Amount)정렬
    df_merged.sort_values('Amount', ascending=False )
    # 상위 2개 꺼내기
    df_top_2 = df_merged.head(2)
    # 상위 18개 저장위한 df에 붙이기
    df_18_selection = pd.concat([df_18_selection,df_top_2], axis=0)

# 실행 확인을 위한 화면 출력
print(df_18_selection.head())

# 결과를 DB에 저장
save_top_18(df_18_selection)
#====================================================================
